In [ ]:
# INSTALACIÓN
!pip install transformers torch accelerate bitsandbytes sentencepiece -q
!pip install pandas beautifulsoup4 lxml -q
!pip install sentence-transformers -q

In [ ]:
# IMPORTS Y MODELOS
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import pandas as pd
import json
import zipfile
import os
from pathlib import Path
import sqlite3
import pandas as pd
from pathlib import Path
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

print("Imports cargados\n")

# LED
print("Cargando LED (Longformer Encoder-Decoder) para resúmenes...")

# LED especializado en papers científicos de arXiv
model_name = "allenai/led-large-16384-arxiv"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16  # Usar FP16 para ahorrar memoria
)

print("LED cargado\n")

# CHATBOT Q&A: SciBER
print("Cargando SciBERT para Q&A...")
qa_pipeline = pipeline(
    "question-answering",
    model="ktrapeznikov/scibert_scivocab_uncased_squad_v2",
    device=0 if torch.cuda.is_available() else -1
)
print("SciBERT cargado\n")

Imports cargados

Cargando LED (Longformer Encoder-Decoder) para resúmenes...


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


SciBERT cargado



In [ ]:
# Crear lista para almacenar resultados
resultados_completos = []

In [ ]:
#FUNCIONES DE EXTRACCIÓN

def extract_from_zip(zip_path, extract_to='./extracted_data'):
    """Descomprime el archivo ZIP"""
    print(f"Descomprimiendo {zip_path}...")

    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print(f" Archivos extraídos a: {extract_to}\n")
    return extract_to


def read_xml_paper(xml_path):
    """Lee un archivo XML y extrae abstract, conclusión, introducción Y todas las demás secciones """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Convertir a texto para usar BeautifulSoup
        xml_string = ET.tostring(root, encoding='unicode')
        soup = BeautifulSoup(xml_string, 'xml')

        sections = {}


        abstract = soup.find('abstract')
        if abstract:

            for tag in abstract.find_all(['kwd-group', 'label', 'title']):
                tag.decompose()

            sections['abstract'] = limpiar_texto_base(abstract.get_text(strip=True, separator=' '))
            print("Abstract encontrado y limpiado")

        # CAPTURAR TODAS LAS SECCIONES
        for section in soup.find_all('sec'):
            title = section.find('title')
            if title:
                title_text = title.get_text().strip()


                section_copy = section

                # Remover elementos que no son contenido
                for tag in section_copy.find_all(['label', 'xref', 'fig', 'table-wrap',
                                                   'disp-formula', 'inline-formula']):
                    tag.decompose()

                # CAMBIO: Limpiar inmediatamente
                section_content = limpiar_texto_base(section_copy.get_text(strip=True, separator=' '))
                title_lower = title_text.lower()

                # Guardar con nombres estándar para las secciones principales
                if any(word in title_lower for word in ['conclusion', 'discussion']):
                    sections['conclusion'] = section_content
                    print(f"  {title_text} encontrado y limpiado")
                elif 'introduction' in title_lower:
                    sections['introduction'] = section_content
                    print(f"  {title_text} encontrado y limpiado")
                elif any(word in title_lower for word in ['method', 'material']):
                    sections['methods'] = section_content
                    print(f"  {title_text} encontrado y limpiado")
                elif any(word in title_lower for word in ['result', 'finding']):
                    sections['results'] = section_content
                    print(f"  {title_text} encontrado y limpiado")

                # Guardar TODAS las secciones con su título original
                sections[f"section_{title_lower}"] = section_content

        body = soup.find('body')
        if body:
            # Remover elementos no deseados del body
            for tag in body.find_all(['kwd-group', 'label', 'xref', 'fig', 'table-wrap',
                                       'disp-formula', 'inline-formula', 'tex-math']):
                tag.decompose()
            # CAMBIO: Limpiar inmediatamente
            sections['full_body'] = limpiar_texto_base(body.get_text(strip=True, separator=' '))
            print("Texto completo del body capturado y limpiado")

        return sections

    except Exception as e:
        print(f"Error leyendo XML: {e}")
        return {}


def get_full_text_from_sections(sections):
    """Concatena todas las secciones en un texto completo estructurado"""
    full_text_parts = []

    # Orden preferido de secciones principales
    priority_order = ['abstract', 'introduction', 'methods', 'results', 'conclusion']

    # Primero agregar secciones principales en orden
    for section_name in priority_order:
        if section_name in sections:
            full_text_parts.append(f"\n\n=== {section_name.upper()} ===\n{sections[section_name]}")

    # Luego agregar otras secciones que comiencen con "section_"
    for section_name, content in sorted(sections.items()):
        if section_name.startswith('section_') and section_name[8:] not in priority_order:
            clean_name = section_name.replace('section_', '').upper()
            full_text_parts.append(f"\n\n=== {clean_name} ===\n{content}")

    return ''.join(full_text_parts) if full_text_parts else sections.get('full_body', '')




def read_json_keywords(json_path, pmc_id=None):
    """Lee archivo JSON y extrae palabras clave para un PMC ID específico"""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        keywords = []
        title = ""

        # Si tenemos PMC ID, buscar específicamente ese artículo
        if pmc_id:
            for item in data:
                if isinstance(item, dict) and item.get('pmc_id') == pmc_id:
                    keywords = item.get('tags', [])
                    title = item.get('title', '')
                    print(f"  ✓ Keywords encontradas para {pmc_id}: {len(keywords)} tags")
                    return keywords, title

        print(f"  No se encontraron keywords para PMC ID: {pmc_id}")
        return [], ""

    except Exception as e:
        print(f" Error leyendo JSON: {e}")
        return [], ""


def combine_sections_for_summary(sections, max_chars=None):
    """
    Combina secciones completas para LED.
    LED puede manejar hasta 16,384 tokens (~65,000 caracteres)
    """
    priority_text = []

    # INCLUIR TODO (LED maneja textos muy largos)

    if 'abstract' in sections:
        priority_text.append(f"Abstract: {sections['abstract']}")

    if 'introduction' in sections:
        priority_text.append(f"Introduction: {sections['introduction']}")

    if 'conclusion' in sections:
        priority_text.append(f"Conclusion: {sections['conclusion']}")

    if 'results' in sections:
        priority_text.append(f"Results: {sections['results']}")

    if 'methods' in sections:
        priority_text.append(f"Methods: {sections['methods']}")

    combined = ' '.join(priority_text)


    if max_chars and len(combined) > max_chars:
        combined = combined[:max_chars]
        print(f"  Texto truncado a {max_chars} caracteres")

    print(f"  Texto combinado: {len(combined)} caracteres")
    return combined


print("Funciones de extracción listas")

Funciones de extracción listas


In [ ]:
def limpiar_texto_base(texto):
    """
    Limpieza BASE aplicable a cualquier texto científico.
    Incluye: Unicode, símbolos, nomenclatura, pero SIN capitalización automática.
    """
    import re
    import html

    if not texto or pd.isna(texto):
        return texto

    texto = str(texto)

    # ========================================================================
    # 1. LIMPIEZA INICIAL - Saltos de línea y espacios
    # ========================================================================
    texto = texto.replace('\r\n', ' ').replace('\n', ' ').replace('\r', ' ')
    texto = texto.replace('\t', ' ')

    # ========================================================================
    # 2. DECODIFICACIÓN - HTML entities y Unicode
    # ========================================================================
    texto = html.unescape(texto)

    # Mapeo completo de Unicode científico
    unicode_map = {
        # Letras griegas
        '\\u03b1': 'α', '\\u0391': 'Α',  # alpha
        '\\u03b2': 'β', '\\u0392': 'Β',  # beta
        '\\u03b3': 'γ', '\\u0393': 'Γ',  # gamma
        '\\u03b4': 'δ', '\\u0394': 'Δ',  # delta
        '\\u03b5': 'ε', '\\u0395': 'Ε',  # epsilon
        '\\u03b6': 'ζ', '\\u0396': 'Ζ',  # zeta
        '\\u03b7': 'η', '\\u0397': 'Η',  # eta
        '\\u03b8': 'θ', '\\u0398': 'Θ',  # theta
        '\\u03ba': 'κ', '\\u039a': 'Κ',  # kappa
        '\\u03bb': 'λ', '\\u039b': 'Λ',  # lambda
        '\\u03bc': 'μ', '\\u039c': 'Μ',  # mu (micro)
        '\\u03bd': 'ν', '\\u039d': 'Ν',  # nu
        '\\u03c0': 'π', '\\u03a0': 'Π',  # pi
        '\\u03c1': 'ρ', '\\u03a1': 'Ρ',  # rho
        '\\u03c3': 'σ', '\\u03a3': 'Σ',  # sigma
        '\\u03c4': 'τ', '\\u03a4': 'Τ',  # tau
        '\\u03c6': 'φ', '\\u03a6': 'Φ',  # phi
        '\\u03c7': 'χ', '\\u03a7': 'Χ',  # chi
        '\\u03c8': 'ψ', '\\u03a8': 'Ψ',  # psi
        '\\u03c9': 'ω', '\\u03a9': 'Ω',  # omega

        # Símbolos matemáticos y científicos
        '\\u00b0': '°',   # grado
        '\\u00b1': '±',   # más/menos
        '\\u00b2': '²',   # superíndice 2
        '\\u00b3': '³',   # superíndice 3
        '\\u00b9': '¹',   # superíndice 1
        '\\u00bc': '¼',   # un cuarto
        '\\u00bd': '½',   # un medio
        '\\u00be': '¾',   # tres cuartos
        '\\u00d7': '×',   # multiplicación
        '\\u00f7': '÷',   # división
        '\\u2032': '′',   # prima
        '\\u2033': '″',   # doble prima
        '\\u2212': '−',   # signo menos
        '\\u2264': '≤',   # menor o igual
        '\\u2265': '≥',   # mayor o igual
        '\\u2260': '≠',   # no igual
        '\\u2248': '≈',   # aproximadamente
        '\\u221e': '∞',   # infinito
        '\\u2013': '–',   # en dash
        '\\u2014': '—',   # em dash
        '\\u2018': ''',   # comilla simple izq
        '\\u2019': ''',   # comilla simple der
        '\\u201c': '"',   # comilla doble izq
        '\\u201d': '"',   # comilla doble der
        '\\u2022': '•',   # bullet
        '\\u2026': '…',   # puntos suspensivos
        '\\u00a0': ' ',   # espacio no rompible

        # Flechas (reacciones químicas)
        '\\u2192': '→',   # flecha derecha
        '\\u2190': '←',   # flecha izquierda
        '\\u2194': '↔',   # flecha doble
        '\\u21d2': '⇒',   # flecha doble derecha
        '\\u21d4': '⇔',   # flecha doble bidireccional

        # Superíndices (0-9)
        '\\u2070': '⁰', '\\u2074': '⁴', '\\u2075': '⁵',
        '\\u2076': '⁶', '\\u2077': '⁷', '\\u2078': '⁸', '\\u2079': '⁹',

        # Subíndices (0-9)
        '\\u2080': '₀', '\\u2081': '₁', '\\u2082': '₂', '\\u2083': '₃',
        '\\u2084': '₄', '\\u2085': '₅', '\\u2086': '₆', '\\u2087': '₇',
        '\\u2088': '₈', '\\u2089': '₉',
    }

    for codigo, simbolo in unicode_map.items():
        texto = texto.replace(codigo, simbolo)

    # ========================================================================
    # 3. NORMALIZACIÓN DE NOMENCLATURA CIENTÍFICA
    # ========================================================================
    texto = re.sub(r'\bsirt1\b', 'SIRT1', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\bsirt2\b', 'SIRT2', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\bsrt2014\b', 'SRT2104', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\be\.?\s*coli\b', 'E. coli', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\bs\.?\s*aureus\b', 'S. aureus', texto, flags=re.IGNORECASE)

    # ========================================================================
    # 4. CORRECCIÓN DE PORCENTAJES Y NÚMEROS
    # ========================================================================
    texto = re.sub(r'(\d)\s*%', r'\1%', texto)
    texto = re.sub(r'(\d)\s*-\s*(\d)', r'\1-\2', texto)
    texto = re.sub(r'(\d)\s*[xX×]\s*10\s*\^\s*(-?\d+)', r'\1×10^\2', texto)

    # ========================================================================
    # 5. CORRECCIÓN DE UNIDADES
    # ========================================================================
    unidades = ['mg', 'μg', 'ng', 'pg', 'g', 'kg', 'mL', 'μL', 'L', 'M', 'mM',
                'μM', 'nM', 'pM', 'mol', 'mmol', 'μmol', 'nm', 'μm', 'mm', 'cm',
                'm', 'km', 's', 'min', 'h', 'Hz', 'kHz', 'MHz', 'V', 'mV', 'A',
                'mA', 'W', 'kW', 'Pa', 'kPa', 'MPa', 'rpm']

    for unidad in unidades:
        texto = re.sub(rf'(\d)({re.escape(unidad)})\b', r'\1 \2', texto)

    # Corregir μ mal escrito
    texto = re.sub(r'\bu([mgLMl])\b', r'μ\1', texto)

    # ========================================================================
    # 6. LIMPIEZA DE PUNTUACIÓN
    # ========================================================================
    texto = re.sub(r'\s+([,.:;!?)\]])', r'\1', texto)
    texto = re.sub(r'([,.:;!?])([^\s\d])', r'\1 \2', texto)
    texto = re.sub(r'\(\s+', '(', texto)
    texto = re.sub(r'\s+\)', ')', texto)
    texto = re.sub(r'\[\s+', '[', texto)
    texto = re.sub(r'\s+\]', ']', texto)
    texto = re.sub(r'\s+', ' ', texto)

    # ========================================================================
    # 7. NORMALIZACIÓN DE ACRÓNIMOS
    # ========================================================================
    texto = re.sub(r'D\s+N\s+A', 'DNA', texto)
    texto = re.sub(r'R\s+N\s+A', 'RNA', texto)
    texto = re.sub(r'P\s+C\s+R', 'PCR', texto)
    texto = re.sub(r'A\s+T\s+P', 'ATP', texto)
    texto = re.sub(r'\bvs\b\.?', 'vs.', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\bet\s+al\.?', 'et al.', texto, flags=re.IGNORECASE)

    # ========================================================================
    # 8. LIMPIEZA DE CARACTERES ESPECIALES
    # ========================================================================
    texto = re.sub(r'[\x00-\x08\x0b-\x0c\x0e-\x1f\x7f-\x9f]', '', texto)
    texto = texto.replace('"', '"').replace('"', '"')
    texto = texto.replace(''', "'").replace(''', "'")
    texto = texto.replace('—', '-').replace('–', '-')
    texto = re.sub(r'\be\.\s*g\.\s*,', 'e.g.,', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\bi\.\s*e\.\s*,', 'i.e.,', texto, flags=re.IGNORECASE)

    # ========================================================================
    # 9. LIMPIEZA FINAL
    # ========================================================================
    texto = texto.strip()

    return texto

In [ ]:
 def limpiar_resumen_corrupto(resumen):
    """
    Limpieza agresiva de código LaTeX, keywords, PACS, DOI, etc.
    """
    import re

    if not resumen:
        return resumen

    original_length = len(resumen)

    # 1. Eliminar desde el primer código PACS/DOI hasta el final
    resumen = re.sub(r'\s+\.?\d+\.\w+.*?(?:doi:\s*10\.\d+/[^\s]+)?\.?\s*$', '', resumen, flags=re.IGNORECASE)

    # 2. Patrón alternativo
    resumen = re.sub(r'\s+\d+\.\d+\.[\w\-\+]+.*$', '', resumen)

    # 3. Eliminar DOI independiente
    resumen = re.sub(r'doi:\s*10\.\d+/[^\s\.]+', '', resumen, flags=re.IGNORECASE)


   # 4. Bloques de keywords (con asteriscos)
    resumen = re.sub(r'\*\s*keywords?\s*:\s*\*.*?(?=\n|$)', '', resumen, flags=re.IGNORECASE)
    resumen = re.sub(r'keywords?\s*:.*?(?=\n|$)', '', resumen, flags=re.IGNORECASE)

    # 5. Key words con lista completa hasta el final
    resumen = re.sub(r'Key words?:.*', '', resumen, flags=re.IGNORECASE | re.DOTALL)

    # 6. NUEVO: Listas largas de términos al final (5+ términos separados por comas)
    resumen = re.sub(r',\s*[a-z\-]+(?:\s+[a-z\-]+)*(?:\s*,\s*[a-z\-]+(?:\s+[a-z\-]+)*){4,}\.?$', '', resumen, flags=re.IGNORECASE)

    # 7. PACS headers
    resumen = re.sub(r'\*\s*pacs\s+numbers?\s*:\s*\*.*?(?=\n|$)', '', resumen, flags=re.IGNORECASE)
    resumen = re.sub(r'pacs\s+numbers?\s*:.*?(?=\n|$)', '', resumen, flags=re.IGNORECASE)
    resumen = re.sub(r'\[\s*,', '', resumen)
    resumen = re.sub(r',\s*\]', '', resumen)
    resumen = re.sub(r'\[\s*\]', '', resumen)
    # 8. Referencias [1,2,3]
    resumen = re.sub(r'\[[\d,\s\-]+\]', '', resumen)

    # 9. Código LaTeX
    resumen = re.sub(r'@xmath\d+', '', resumen)
    resumen = re.sub(r'\$.*?\$', '', resumen)

    # 10. Símbolos sueltos
    resumen = re.sub(r'\*+', '', resumen)
    resumen = re.sub(r'\+\s*\+', '', resumen)

    # 11. Afiliaciones
    resumen = re.sub(r'department of.*?usa\.?', '', resumen, flags=re.IGNORECASE)

    # 12. Comillas
    resumen = re.sub(r"[`''\"<>]", '', resumen)

    # 13. Espacios y saltos de línea
    resumen = re.sub(r'\n+', ' ', resumen)
    resumen = re.sub(r'\s+', ' ', resumen)

    # 14. Espacios antes de puntuación
    resumen = re.sub(r'\s+([.,;:!?])', r'\1', resumen)

    # 15. Puntos y comas múltiples
    resumen = re.sub(r'\.{2,}', '.', resumen)
    resumen = re.sub(r',{2,}', ',', resumen)

    resumen = resumen.strip()

    # 16. Quitar punto inicial
    if resumen.startswith('.'):
        resumen = resumen[1:].strip()

    # 17. Quitar coma final antes del punto
    resumen = re.sub(r',\s*\.$', '.', resumen)

    if len(resumen) < original_length * 0.8:
        print(f"      [LIMPIEZA] Eliminados {original_length - len(resumen)} caracteres")

    return resumen

In [ ]:
def generate_summary_led(texto, titulo="", keywords=None):
    """Genera resumen con LED (Longformer Encoder-Decoder)"""

    input_text = texto

    if titulo:
        input_text = f"Title: {titulo}\n\n{input_text}"

    if keywords:
        keywords_str = ', '.join(keywords[:10])
        input_text = f"Keywords: {keywords_str}\n\n{input_text}"

    print("   Generando resumen con LED...")
    print(f"   Longitud del texto: {len(input_text)} caracteres")

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=16384,
        truncation=True,
        padding=True
    ).to(model.device)

    global_attention_mask = torch.zeros_like(inputs['input_ids'])
    global_attention_mask[:, 0] = 1

    with torch.no_grad():
        summary_ids = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            global_attention_mask=global_attention_mask,
            max_length=int(450),
            min_length=int(180),
            num_beams=int(4),
            length_penalty=float(2.0),
            no_repeat_ngram_size=int(3),
            early_stopping=True

        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(f"   Resumen generado: {len(summary)} caracteres")

    # LIMPIEZA COMPLETA
    import re


    summary = limpiar_resumen_corrupto(summary)

    # Capitalización
    summary = re.sub(r'\.\s+([a-z])', lambda m: '. ' + m.group(1).upper(), summary)
    if summary:
        summary = summary[0].upper() + summary[1:] if len(summary) > 1 else summary.upper()


    if summary and summary[-1] not in '.!?':
        summary = summary.rstrip()

        last_period = summary.rfind('.')
        if last_period > len(summary) * 0.7:
            summary = summary[:last_period + 1]
        else:
            summary += '.'


    summary = summary.strip()
    summary = re.sub(r'\s+', ' ', summary)
    summary = re.sub(r'\.{2,}', '.', summary)

    return summary

In [ ]:
# PROCESAMIENTO PRINCIPAL
def process_paper_from_files(xml_path, json_path, titulo="Artículo", link="", paper_id=None):
    """Procesa un paper desde archivos XML y JSON locales"""

    print(f"\n{'='*70}")
    print(f"Procesando: {titulo[:50]}...")
    print(f"PMC ID: {paper_id}")
    print(f"{'='*70}\n")

    print("Extrayendo datos del XML...")
    sections = read_xml_paper(xml_path)

    if not sections:
        print("No se pudo extraer información del XML")
        return None

    print("\nExtrayendo keywords del JSON...")
    keywords_json, title_json = read_json_keywords(json_path, pmc_id=paper_id)

    # Ya NO necesitas limpiar aquí
    texto_completo_estructurado = get_full_text_from_sections(sections)
    texto_combinado = combine_sections_for_summary(sections, max_chars=None)

    print("\nGenerando resumen con LED (Longformer)...")
    resumen_limpio = generate_summary_led(texto_combinado, titulo, keywords_json)

    # ELIMINADO: limpiar_texto_completo() - ya viene limpio

    resultado = {
        'pmc_id': paper_id,
        'titulo': titulo,
        'link': link,
        'resumen': resumen_limpio,
        'keywords': ', '.join(keywords_json) if keywords_json else 'No keywords found',
        'xml_path': xml_path,
        'json_path': json_path,
        'texto_completo': texto_completo_estructurado,  # Ya viene limpio
        'texto_para_chatbot': texto_combinado  # Ya viene limpio
    }

    print(f"\n{'='*70}")
    print(f"RESULTADOS")
    print(f"{'='*70}")
    print(f"\nTITULO:\n{titulo}")
    print(f"\nLINK:\n{link}")
    print(f"\nPMC ID:\n{paper_id}")

    if keywords_json:
        print(f"\nKEYWORDS (del JSON):\n{', '.join(keywords_json)}")

    print(f"\nRESUMEN (LED - LIMPIO):")
    print(f"{'-'*70}")
    print(resumen_limpio[:500] + "..." if len(resumen_limpio) > 500 else resumen_limpio)
    print(f"{'-'*70}\n")

    print(f"TEXTO COMPLETO: {len(texto_completo_estructurado)} caracteres (limpio)")

    return resultado

In [ ]:
# FUNCIÓN PARA PROCESAR TODOS LOS XMLs

def process_all_xmls_in_folder(xml_folder, json_path):
    """Procesa TODOS los XMLs usando únicamente XML + JSON"""

    # Listar todos los XMLs
    xml_files = [f for f in os.listdir(xml_folder) if f.endswith('.xml')]
    print(f"\nArchivos XML encontrados: {len(xml_files)}")

    # Cargar JSON completo
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    # Crear mapeo desde JSON
    json_map = {}
    for item in json_data:
        if isinstance(item, dict) and 'pmc_id' in item:
            pmc_id = item['pmc_id']
            json_map[pmc_id] = {
                'titulo': item.get('title', ''),
                'keywords': item.get('tags', [])
            }

    print(f"JSON: {len(json_map)} articulos mapeados")

    # VALIDACION CRUZADA
    print(f"\nVALIDACION:")
    solo_xml = set([f.replace('.xml', '') for f in xml_files])
    solo_json = set(json_map.keys())

    en_ambos = solo_xml & solo_json
    solo_en_xml = solo_xml - solo_json
    solo_en_json = solo_json - solo_xml

    print(f"  - En XML y JSON: {len(en_ambos)}")
    print(f"  - Solo en XML (sin metadata): {len(solo_en_xml)}")
    print(f"  - Solo en JSON (sin archivo): {len(solo_en_json)}")

    if solo_en_xml:
        print(f"    Ejemplos sin metadata: {list(solo_en_xml)[:3]}")

    resultados = []

    for idx, xml_file in enumerate(xml_files[:], 1):  # CAMBIAR LIMITE AQUI
        paper_id = xml_file.replace('.xml', '')
        xml_path = os.path.join(xml_folder, xml_file)

        # Obtener datos del JSON
        json_info = json_map.get(paper_id, {})
        titulo = json_info.get('titulo', f"Paper {paper_id}")
        keywords = json_info.get('keywords', [])

        # Construir link estándar
        link = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{paper_id}/"

        print(f"\n{'='*70}")
        print(f"[{idx}/{len(xml_files)}] {paper_id}")
        print(f"Titulo: {titulo[:60]}...")
        print(f"Link: {link}")
        print(f"{'='*70}")

        try:
            resultado = process_paper_from_files(
                xml_path=xml_path,
                json_path=json_path,
                titulo=titulo,
                link=link,
                paper_id=paper_id
            )

            if resultado:
                resultados.append({
                    'PMC_ID': resultado['pmc_id'],
                    'Titulo': resultado['titulo'],
                    'Link': resultado['link'],
                    'Resumen': resultado['resumen'],
                    'Keywords': resultado['keywords'],
                    'Texto_Completo': resultado['texto_completo']
                })

        except Exception as e:
            print(f"Error procesando {paper_id}: {str(e)}")
            continue

    print(f"\nProcesamiento completado: {len(resultados)}/{len(xml_files)} papers")
    return resultados

In [ ]:
def save_results_to_csv(resultados, output_path='resultados_resumenes.csv'):
    """Guarda los resultados en un CSV - resumenes ya vienen limpios"""

    # Crear DataFrame directamente desde los diccionarios
    # Esto toma TODAS las columnas que vengan en resultados
    df = pd.DataFrame(resultados)

    # Verificar que las columnas esperadas existan
    columnas_esperadas = ['PMC_ID', 'Titulo', 'Link', 'Resumen', 'Keywords', 'Texto_Completo']
    columnas_faltantes = [col for col in columnas_esperadas if col not in df.columns]

    if columnas_faltantes:
        print(f"\nADVERTENCIA: Faltan columnas: {columnas_faltantes}")

    # Reordenar columnas si todas existen
    if not columnas_faltantes:
        df = df[columnas_esperadas]

    df.to_csv(
        output_path,
        index=False,
        encoding='utf-8',
        quoting=1
    )

    print(f"\n{'='*70}")
    print(f"DATOS GUARDADOS")
    print(f"{'='*70}")
    print(f"Archivo: {output_path}")
    print(f"Total de papers procesados: {len(df)}")
    print(f"\nColumnas guardadas (en orden):")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i}. {col}")

    if 'Texto_Completo' in df.columns:
        avg_length = df['Texto_Completo'].str.len().mean()
        print(f"\nLongitud promedio de Texto_Completo: {avg_length:.0f} caracteres")

    print(f"\nBase de datos completa y lista!")

    return df

In [ ]:
def diagnosticar_base_datos(xml_folder, json_path):
    """Diagnostico completo usando solo XML y JSON"""

    print("\n" + "="*70)
    print("DIAGNOSTICO DE BASE DE DATOS")
    print("="*70)

    # 1. XMLs
    xml_files = [f.replace('.xml', '') for f in os.listdir(xml_folder) if f.endswith('.xml')]
    print(f"\n1. ARCHIVOS XML: {len(xml_files)}")
    print(f"   Primeros 5: {xml_files[:5]}")

    # 2. JSON
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    pmc_ids_json = [item.get('pmc_id') for item in json_data if isinstance(item, dict)]
    print(f"\n2. JSON: {len(pmc_ids_json)} articulos")
    print(f"   Primeros 5: {pmc_ids_json[:5]}")

    # 3. INTERSECCIONES
    set_xml = set(xml_files)
    set_json = set(pmc_ids_json)

    print(f"\n3. INTERSECCIONES:")
    print(f"   XML con JSON: {len(set_xml & set_json)}")

    # 4. DIFERENCIAS
    print(f"\n4. ELEMENTOS UNICOS:")
    solo_xml = set_xml - set_json
    solo_json = set_json - set_xml

    print(f"   Solo en XML (sin metadata JSON): {len(solo_xml)}")
    if solo_xml:
        print(f"      Ejemplos: {list(solo_xml)[:5]}")

    print(f"   Solo en JSON (sin archivo XML): {len(solo_json)}")
    if solo_json:
        print(f"      Ejemplos: {list(solo_json)[:5]}")

    # 5. VALIDAR TITULOS de archivos comunes
    print(f"\n5. VALIDACION DE TITULOS (JSON vs XML):")

    comunes = list(set_xml & set_json)[:3]
    titulos_coinciden = 0
    titulos_difieren = 0

    for pmc_id in comunes:
        # Titulo del JSON
        titulo_json = next((item['title'] for item in json_data if item.get('pmc_id') == pmc_id), None)

        # Titulo del XML
        xml_path = os.path.join(xml_folder, f"{pmc_id}.xml")
        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()
            xml_string = ET.tostring(root, encoding='unicode')
            soup = BeautifulSoup(xml_string, 'xml')

            # Buscar titulo en el XML
            title_tag = soup.find('article-title')
            titulo_xml = title_tag.get_text(strip=True) if title_tag else None

            coincide = titulo_json == titulo_xml
            if coincide:
                titulos_coinciden += 1
            else:
                titulos_difieren += 1

            print(f"\n   {pmc_id}:")
            print(f"      JSON: {titulo_json[:50] if titulo_json else 'N/A'}...")
            print(f"      XML:  {titulo_xml[:50] if titulo_xml else 'N/A'}...")
            print(f"      Match: {'SI' if coincide else 'NO'}")

        except Exception as e:
            print(f"\n   {pmc_id}: Error leyendo XML - {e}")

    print(f"\n6. RESUMEN:")
    print(f"   Titulos que coinciden: {titulos_coinciden}")
    print(f"   Titulos que difieren: {titulos_difieren}")
    print(f"   DECISION: Usar titulos del JSON (metadata oficial)")

    return {
        'xml_count': len(xml_files),
        'json_count': len(pmc_ids_json),
        'comunes': len(set_xml & set_json)
    }

In [ ]:
def validar_base_datos(df):
    """Valida la calidad de los datos guardados"""

    print("\n" + "="*70)
    print("VALIDACION DE DATOS GUARDADOS")
    print("="*70)

    problemas = []

    # 1. Verificar columnas requeridas
    columnas_requeridas = ['PMC_ID', 'Titulo', 'Link', 'Resumen', 'Keywords', 'Texto_Completo']
    for col in columnas_requeridas:
        if col not in df.columns:
            problemas.append(f"Falta columna: {col}")

    # 2. Verificar valores nulos
    print("\n1. VALORES NULOS:")
    for col in df.columns:
        nulos = df[col].isna().sum()
        if nulos > 0:
            print(f"   {col}: {nulos} nulos")
            problemas.append(f"{col} tiene {nulos} valores nulos")

    if df.isna().sum().sum() == 0:
        print("   Sin valores nulos")

    # 3. Verificar duplicados
    print(f"\n2. DUPLICADOS:")
    duplicados = df['PMC_ID'].duplicated().sum()
    if duplicados > 0:
        print(f"   PMC_IDs duplicados: {duplicados}")
        problemas.append(f"{duplicados} PMC_IDs duplicados")
    else:
        print("   Sin duplicados")

    # 4. Verificar longitudes
    print(f"\n3. LONGITUDES:")
    print(f"   Resumenes:")
    print(f"      Promedio: {df['Resumen'].str.len().mean():.0f} caracteres")
    print(f"      Minimo: {df['Resumen'].str.len().min():.0f} caracteres")
    print(f"      Maximo: {df['Resumen'].str.len().max():.0f} caracteres")

    resumenes_muy_cortos = (df['Resumen'].str.len() < 50).sum()
    if resumenes_muy_cortos > 0:
        print(f"      ADVERTENCIA: {resumenes_muy_cortos} resumenes < 50 caracteres")
        problemas.append(f"{resumenes_muy_cortos} resumenes muy cortos")

    print(f"   Texto_Completo:")
    print(f"      Promedio: {df['Texto_Completo'].str.len().mean():.0f} caracteres")
    print(f"      Minimo: {df['Texto_Completo'].str.len().min():.0f} caracteres")
    print(f"      Maximo: {df['Texto_Completo'].str.len().max():.0f} caracteres")

    textos_muy_cortos = (df['Texto_Completo'].str.len() < 500).sum()
    if textos_muy_cortos > 0:
        print(f"      ADVERTENCIA: {textos_muy_cortos} textos < 500 caracteres")
        problemas.append(f"{textos_muy_cortos} textos completos muy cortos")

    # 5. Verificar keywords
    print(f"\n4. KEYWORDS:")
    sin_keywords = (df['Keywords'] == 'No keywords found').sum()
    print(f"   Sin keywords: {sin_keywords} articulos")
    if sin_keywords == len(df):
        problemas.append("NINGUN articulo tiene keywords")

    # 6. Verificar formato de links
    print(f"\n5. LINKS:")
    links_validos = df['Link'].str.contains('ncbi.nlm.nih.gov/pmc', na=False).sum()
    print(f"   Links validos: {links_validos}/{len(df)}")
    if links_validos < len(df):
        problemas.append(f"{len(df) - links_validos} links con formato incorrecto")

    # RESUMEN FINAL
    print(f"\n" + "="*70)
    if not problemas:
        print("VALIDACION: EXITOSA - Sin problemas detectados")
    else:
        print(f"VALIDACION: CON ADVERTENCIAS - {len(problemas)} problemas detectados:")
        for i, problema in enumerate(problemas, 1):
            print(f"   {i}. {problema}")
    print("="*70)

    return len(problemas) == 0

In [ ]:
def validar_calidad_resumenes(df):
    """Detecta resúmenes con problemas"""

    print("\nVALIDACION DE CALIDAD DE RESUMENES:")
    print("="*70)

    problemas = []

    for idx, row in df.iterrows():
        resumen = row['Resumen']
        pmc_id = row['PMC_ID']

        # Detectar código LaTeX
        if '@xmath' in resumen or '#' in resumen[:100]:
            problemas.append({
                'PMC_ID': pmc_id,
                'Problema': 'Código LaTeX/markup',
                'Muestra': resumen[:100]
            })

        # Detectar keywords duplicadas
        if 'keywords:' in resumen.lower():
            problemas.append({
                'PMC_ID': pmc_id,
                'Problema': 'Keywords duplicadas',
                'Muestra': resumen[:100]
            })

        # Detectar afiliaciones
        if 'department of' in resumen.lower():
            problemas.append({
                'PMC_ID': pmc_id,
                'Problema': 'Afiliaciones en resumen',
                'Muestra': resumen[:100]
            })

    if problemas:
        print(f"\nSe encontraron {len(problemas)} resumenes con problemas:")
        for p in problemas[:5]:  # Mostrar primeros 5
            print(f"\n  PMC_ID: {p['PMC_ID']}")
            print(f"  Problema: {p['Problema']}")
            print(f"  Muestra: {p['Muestra']}...")

        # Guardar reporte
        df_problemas = pd.DataFrame(problemas)
        df_problemas.to_csv('resumenes_con_problemas.csv', index=False)
        print(f"\nReporte completo guardado en: resumenes_con_problemas.csv")
    else:
        print("\nTodos los resumenes tienen buena calidad")

    return len(problemas) == 0

In [ ]:

zip_path = '/content/datasets-20251004T053312Z-1-001.zip'  # Ajusta la ruta
extracted_path = extract_from_zip(zip_path)

Descomprimiendo /content/datasets-20251004T053312Z-1-001.zip...
 Archivos extraídos a: ./extracted_data



In [ ]:
def process_and_save_all():
    """Procesa todos los articulos usando solo XML y JSON"""

    xml_folder = '/content/extracted_data/datasets/xml/sb'
    json_path = '/content/extracted_data/datasets/json/sb.json'

    # PASO 1: DIAGNOSTICO PREVIO
    print("\n" + "="*70)
    print("PASO 1: DIAGNOSTICO DE BASE DE DATOS")
    print("="*70)

    diagnostico = diagnosticar_base_datos(xml_folder, json_path)

    if diagnostico['comunes'] == 0:
        print("\nERROR: No hay articulos en comun entre XML y JSON")
        print("   Revisa los archivos antes de continuar.")
        return None

    print(f"\n{diagnostico['comunes']} articulos listos para procesar")
    print(f"   (De {diagnostico['xml_count']} XMLs disponibles)")

    # PASO 2: PROCESAMIENTO
    print("\n" + "="*70)
    print("PASO 2: PROCESANDO ARTICULOS...")
    print("="*70)

    resultados = process_all_xmls_in_folder(xml_folder, json_path)

    if not resultados:
        print("\nNo se procesaron articulos. Revisa los errores arriba.")
        return None

    # PASO 3: GUARDAR RESULTADOS
    print("\n" + "="*70)
    print("PASO 3: GUARDANDO RESULTADOS...")
    print("="*70)

    # Guardar en CSV
    output_file = 'base_datos_articulos_completa.csv'
    df_final = save_results_to_csv(resultados, output_file)

    # Guardar en JSON
    json_output = 'base_datos_articulos_completa.json'
    df_final.to_json(json_output, orient='records', indent=2, force_ascii=False)
    print(f"Tambien guardado en: {json_output}")

    # PASO 4: VALIDAR DATOS GUARDADOS
    print("\n" + "="*70)
    print("PASO 4: VALIDANDO DATOS...")
    print("="*70)

    # AQUI VAN LAS DOS VALIDACIONES:
    validacion_exitosa = validar_base_datos(df_final)
    validacion_resumenes = validar_calidad_resumenes(df_final)

    # PASO 5: REPORTE FINAL
    print("\n" + "="*70)
    print("PASO 5: REPORTE FINAL")
    print("="*70)

    print(f"\nProcesamiento exitoso:")
    print(f"   - Articulos procesados: {len(resultados)}")
    print(f"   - Archivos generados:")
    print(f"      * {output_file}")
    print(f"      * {json_output}")

    # Muestra de datos
    print(f"\nPrimeras 2 filas:")
    print(df_final[['PMC_ID', 'Titulo']].head(2).to_string(index=False))

    # Muestra del primer articulo completo
    print(f"\nMuestra del primer articulo:")
    primer_articulo = df_final.iloc[0]
    print(f"   PMC_ID: {primer_articulo['PMC_ID']}")
    print(f"   Titulo: {primer_articulo['Titulo'][:60]}...")
    print(f"   Link: {primer_articulo['Link']}")
    print(f"   Keywords: {primer_articulo['Keywords'][:80]}...")
    print(f"   Resumen ({len(primer_articulo['Resumen'])} chars):")
    print(f"   {primer_articulo['Resumen'][:200]}...")
    print(f"   Texto_Completo ({len(primer_articulo['Texto_Completo'])} chars):")
    print(f"   {primer_articulo['Texto_Completo'][:300]}...")

    print(f"\nBase de datos completa y lista para usar!")
    print(f"   Puedes cargarla con: pd.read_csv('{output_file}')")

    return df_final

In [ ]:


print("\n" + "="*70)
print("INICIANDO PROCESAMIENTO COMPLETO")
print("FUENTES: XML + JSON ")
print("="*70)

# Ejecutar todo el pipeline
df_completo = process_and_save_all()

# Verificar resultado
if df_completo is not None:
    print("\n" + "="*70)
    print("PROCESAMIENTO COMPLETADO EXITOSAMENTE")
    print("="*70)
    print(f"\nResumen final:")
    print(f"   - Total de articulos: {len(df_completo)}")
    print(f"   - Columnas: {', '.join(df_completo.columns)}")

    # VERIFICACION DE LIMPIEZA
    print("\n" + "="*70)
    print("VERIFICACION DE CALIDAD DE LIMPIEZA")
    print("="*70)

    # Verificar primer artículo
    primer_articulo = df_completo.iloc[0]

    # Checks de limpieza
    tiene_unicode_crudo = '\\u' in str(primer_articulo['Texto_Completo'])
    tiene_latex = '@xmath' in str(primer_articulo['Resumen'])
    tiene_html = '&nbsp;' in str(primer_articulo['Texto_Completo']) or '&amp;' in str(primer_articulo['Texto_Completo'])

    print(f"\nAnálisis del primer artículo ({primer_articulo['PMC_ID']}):")
    print(f"   - Unicode crudo (\\u): {'SI - PROBLEMA' if tiene_unicode_crudo else 'NO - OK'}")
    print(f"   - Código LaTeX (@xmath): {'SI - PROBLEMA' if tiene_latex else 'NO - OK'}")
    print(f"   - Entidades HTML: {'SI - PROBLEMA' if tiene_html else 'NO - OK'}")
    print(f"   - Longitud resumen: {len(primer_articulo['Resumen'])} chars")
    print(f"   - Longitud texto completo: {len(primer_articulo['Texto_Completo'])} chars")

    # Muestra de resumen
    print(f"\nMuestra de resumen limpio:")
    print(f"{'-'*70}")
    print(primer_articulo['Resumen'][:300])
    print(f"{'-'*70}")

    # Verificación masiva de toda la base de datos
    print(f"\nVerificación masiva de {len(df_completo)} artículos:")
    total_unicode = df_completo['Texto_Completo'].astype(str).str.contains('\\\\u').sum()
    total_latex = df_completo['Resumen'].astype(str).str.contains('@xmath').sum()
    total_html = df_completo['Texto_Completo'].astype(str).str.contains('&nbsp;|&amp;').sum()

    print(f"   - Artículos con Unicode crudo: {total_unicode}")
    print(f"   - Artículos con LaTeX en resumen: {total_latex}")
    print(f"   - Artículos con HTML sin limpiar: {total_html}")

    if total_unicode == 0 and total_latex == 0 and total_html == 0:
        print("\n*** LIMPIEZA EXITOSA: Todos los textos están limpios ***")
    else:
        print("\n*** ADVERTENCIA: Algunos textos tienen problemas de limpieza ***")

    print(f"\nProximos pasos:")
    print(f"   1. Si la limpieza es exitosa, continúa con el análisis")
    print(f"   2. Probar el chatbot con:")
    print(f"      interactive_chat(df_completo['Texto_Completo'].iloc[0], df_completo['Titulo'].iloc[0])")
else:
    print("\n" + "="*70)
    print("ERROR EN EL PROCESAMIENTO")
    print("="*70)
    print("\nRevisa los mensajes de error arriba")


INICIANDO PROCESAMIENTO COMPLETO
FUENTES: XML + JSON 

PASO 1: DIAGNOSTICO DE BASE DE DATOS

DIAGNOSTICO DE BASE DE DATOS

1. ARCHIVOS XML: 571
   Primeros 5: ['PMC3430326', 'PMC7414185', 'PMC3337602', 'PMC9964234', 'PMC11362537']

2. JSON: 571 articulos
   Primeros 5: ['PMC5580210', 'PMC12040010', 'PMC9617909', 'PMC8323809', 'PMC3851762']

3. INTERSECCIONES:
   XML con JSON: 571

4. ELEMENTOS UNICOS:
   Solo en XML (sin metadata JSON): 0
   Solo en JSON (sin archivo XML): 0

5. VALIDACION DE TITULOS (JSON vs XML):

   PMC8816950:
      JSON: Production of reactive oxygen species and involvem...
      XML:  Production of reactive oxygen species and involvem...
      Match: NO

   PMC4776492:
      JSON: REDUCED CHLOROPLAST COVERAGE genes from Arabidopsi...
      XML:  REDUCED CHLOROPLAST COVERAGEgenes fromArabidopsis ...
      Match: NO

   PMC10503492:
      JSON: Artificial Gravity Partially Protects Space-induce...
      XML:  Artificial Gravity Partially Protects Space-induce...
 

Input ids are automatically padded from 259 to 1024 to be a multiple of `config.attention_window`: 1024


   Resumen generado: 939 caracteres
      [LIMPIEZA] Eliminados 209 caracteres

RESULTADOS

TITULO:
The MscS and MscL Families of Mechanosensitive Channels Act as Microbial Emergency Release Valves

LINK:
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3430326/

PMC ID:
PMC3430326

KEYWORDS (del JSON):
minireview

RESUMEN (LED - LIMPIO):
----------------------------------------------------------------------
Most single - cell organisms appear to possess members of one or both families of bacterial mechanosensitive channels, which can act as biological emergency release valves that allow cytoplasmic solutes to be jettisoned rapidly from the cell. While this is undoubtedly a function of these proteins, the discovery of the presence of mscS homologues in plant organelles and mscL in fungus and mycoplasma genomes may complicate this simplistic interpretation of the physiology underlying these proteins....
----------------------------------------------------------------------

TEXTO COMPLETO: 

Input ids are automatically padded from 5616 to 6144 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 2503 to 3072 to be a multiple of `config.attention_window`: 1024


   Resumen generado: 1130 caracteres

RESULTADOS

TITULO:
Tonoplast-localized Ca2+ pumps regulate Ca2+ signals during pattern-triggered immunity in Arabidopsis thaliana

LINK:
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7414185/

PMC ID:
PMC7414185

KEYWORDS (del JSON):
autoinhibited calcium atpase, calcium, flg22, vacuole

RESUMEN (LED - LIMPIO):
----------------------------------------------------------------------
We show that tonoplast-localized calcium atpase ( ca 2+ ) pumps play an important role in regulating both response to the bacterial elicitor peptide flg22 and in the induction of pattern-triggered immunity ( ptd ). We show using direct measurements and confocal ratio imaging that when the bacterial peptide is used to trigger immune responses in mutants missing the plasma membrane cotyledon calcium pump ( col-0 ), it causes disrupted calcium 2+ signals that are higher in amplitude than expected f...
----------------------------------------------------------------------

TE

Input ids are automatically padded from 1931 to 2048 to be a multiple of `config.attention_window`: 1024


   Resumen generado: 1304 caracteres

RESULTADOS

TITULO:
Designing a Novel Monitoring Approach for the Effects of Space Travel on Astronauts’ Health

LINK:
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9964234/

PMC ID:
PMC9964234

KEYWORDS (del JSON):
space biology, diagnostic assay, health, transcriptome, space flight

RESUMEN (LED - LIMPIO):
----------------------------------------------------------------------
Space exploration and extraterrestrial civilization have fascinated humankind since the earliest days of human history. However, it was only in the last century that humankind finally took a solid step forward by sending astronauts into space and landing on the moon. In the 21 st century, humankind has made another great leap forward in space exploration: expediting the International space station (ISS ) program sending unmanned rovers to seek signs of life on Mars ] and starting private space v...
----------------------------------------------------------------------

TEXTO 

Input ids are automatically padded from 13128 to 13312 to be a multiple of `config.attention_window`: 1024


Texto completo del body capturado y limpiado

Extrayendo keywords del JSON...
  ✓ Keywords encontradas para PMC11362537: 4 tags
  Texto combinado: 57488 caracteres

Generando resumen con LED (Longformer)...
   Generando resumen con LED...
   Longitud del texto: 57627 caracteres
   Resumen generado: 1134 caracteres

RESULTADOS

TITULO:
Spaceflight alters host-gut microbiota interactions

LINK:
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11362537/

PMC ID:
PMC11362537

KEYWORDS (del JSON):
microbial communities, microbial genetics, metagenomics, microbiota

RESUMEN (LED - LIMPIO):
----------------------------------------------------------------------
Metagenomic assessment of the murine gut microbiome reveals significant spaceflight-associated changes in bacteria linked to bile and fatty acid metabolism. These changes in relative abundance were largely consistent in two groups of mice after 29 and 56 days of spaceflight when compared to different on-Earth control groups at different tim